# Setting Up

## load modules

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
import itertools as itt
import os
import sys

import holoviews as hv
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade, regrid
from holoviews.util import Dynamic
from IPython.core.display import display

## set path and parameters

In [ ]:
# Set up Initial Basic Parameters#
minian_path = "."
dpath = "../data/m16/2022_10_20/17_58_52/miniscope_top/"
intpath = "~/var/2s_validation/minian_intermediate"
intpath = os.path.normpath(os.path.expanduser(intpath))
minian_ds_path = intpath
subset = dict(frame=slice(0, 4999))
subset_mc = None
interactive = True
output_size = 100
n_workers = int(os.getenv("MINIAN_NWORKERS", 4))
param_save_minian = {
    "dpath": minian_ds_path,
    "overwrite": True,
}

# Pre-processing Parameters#
param_load_videos = {
    "pattern": "[0-9]+\.avi$",
    "dtype": np.uint8,
    "downsample": dict(frame=1, height=1, width=1),
    "downsample_strategy": "subset",
}
param_glow_rm = {"method": "uniform", "wnd": 80}
param_denoise = {"method": "median", "ksize": 3}
param_background_removal = {"method": "tophat", "wnd": 5}

# Motion Correction Parameters#
subset_mc = None
param_estimate_motion = {"dim": "frame", 'alt_error': None, 'upsample': 10}

# seed params
param_find_seed = {
    "wnd_k0": 8,
    "wnd_k1": 15,
    "diff_thres": 8
}
param_constructA = {
    "min_size": 64,
    "max_size": 400,
    "pad": 2
}
param_mergeA = {
    "cos_thres": 0.5
}

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MINIAN_INTERMEDIATE"] = intpath

## import minian

In [ ]:
%%capture
sys.path.append(minian_path)
from minian.cnmf import (
    compute_AtC,
    compute_trace,
    get_noise_fft,
    smooth_sig,
    unit_merge,
    update_spatial,
    update_temporal,
    update_background,
)
from minian.initialization import (
    gmm_refine,
    initA,
    initC,
    intensity_refine,
    ks_refine,
    pnr_refine,
    seeds_init,
    seeds_merge,
)
from minian.motion_correction import apply_transform, estimate_motion
from minian.preprocessing import denoise, remove_background
from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)
from minian.visualization import (
    CNMFViewer,
    VArrayViewer,
    generate_videos,
    visualize_gmm_fit,
    visualize_motion,
    visualize_preprocess,
    visualize_seeds,
    visualize_spatial_update,
    visualize_temporal_update,
    write_video,
)
from routine.static_channel import constructA, find_seed, mergeA
from routine.plotting import plotA_contour

## module initialization

In [ ]:
dpath = os.path.abspath(dpath)
hv.notebook_extension("bokeh", width=100)

## start cluster

In [ ]:
cluster = LocalCluster(
    n_workers=8,
    memory_limit="4GB",
    resources={"MEM": 1},
    threads_per_worker=2,
    dashboard_address=":23456",
)
annt_plugin = TaskAnnotation()
cluster.scheduler.add_plugin(annt_plugin)
client = Client(cluster)

# Pre-processing

## loading videos and visualization

In [ ]:
varr = load_videos(dpath, **param_load_videos).sel(**subset)
chk, _ = get_optimal_chk(varr, dtype=float)

In [ ]:
%%time
varr = save_minian(
    varr.chunk({"frame": chk["frame"], "height": -1, "width": -1}).rename("varr"),
    intpath,
    overwrite=True,
)

## visualize raw data and optionally set roi for motion correction

In [ ]:
hv.output(size=output_size)
if interactive:
    vaviewer = VArrayViewer(varr, framerate=5, summary=["mean", "max"])
    display(vaviewer.show())

## subset part of video

In [ ]:
varr_ref = varr

## glow removal and visualization

In [ ]:
varr_ref = (
    remove_background(varr_ref.astype(float), **param_glow_rm)
    .clip(0, 255)
    .astype(np.uint8)
)

In [ ]:
hv.output(size=int(output_size * 0.7))
if interactive:
    vaviewer = VArrayViewer(
        [varr.rename("original"), varr_ref.rename("glow_removed")],
        framerate=5,
        summary=None,
        layout=True,
    )
    display(vaviewer.show())

## denoise

In [ ]:
hv.output(size=int(output_size * 0.6))
if interactive:
    display(
        visualize_preprocess(
            varr_ref.isel(frame=50).compute(),
            denoise,
            method=["median"],
            ksize=[3, 5, 7],
        )
    )

The following cell would carry out denoise step.
Be sure to [change the parameters](https://minian.readthedocs.io/page/start_guide/faq.html#i-don-t-know-python-can-i-still-use-the-pipeline) based on visualization results before running the following cell.

In [ ]:
varr_ref = denoise(varr_ref, **param_denoise)

## background removal

In [ ]:
hv.output(size=int(output_size * 0.6))
if interactive:
    display(
        visualize_preprocess(
            varr_ref.isel(frame=50).compute(),
            remove_background,
            method=["tophat"],
            wnd=[5, 10, 15],
        )
    )

The following cell would carry out background removal step.
Be sure to [change the parameters](https://minian.readthedocs.io/page/start_guide/faq.html#i-don-t-know-python-can-i-still-use-the-pipeline) based on visualization results before running the following cell.

In [ ]:
varr_ref = remove_background(varr_ref, **param_background_removal)

## save result

In [ ]:
%%time
varr_ref = save_minian(varr_ref.rename("varr_ref"), dpath=intpath, overwrite=True)

# Motion Correction

## estimate motion

In [ ]:
%%time
motion = estimate_motion(varr_ref.sel(subset_mc), **param_estimate_motion)

## save motion

In [ ]:
%%time
motion = save_minian(
    motion.rename("motion").chunk({"frame": chk["frame"]}), **param_save_minian
)

## visualization of motion

In [ ]:
hv.output(size=output_size)
visualize_motion(motion)

## apply transform

In [ ]:
Y = apply_transform(varr_ref, motion, fill=0)

## save result

In [ ]:
%%time
Y_fm_chk = save_minian(Y.astype(float).rename("Y_fm_chk"), intpath, overwrite=True)
Y_hw_chk = save_minian(
    Y_fm_chk.rename("Y_hw_chk"),
    intpath,
    overwrite=True,
    chunks={"frame": -1, "height": chk["height"], "width": chk["width"]},
)

## visualization of motion-correction

In [ ]:
im_opts = dict(
    frame_width=500,
    aspect=varr_ref.sizes["width"] / varr_ref.sizes["height"],
    cmap="Viridis",
    colorbar=True,
)
(
    regrid(
        hv.Image(
            varr_ref.max("frame").compute().astype(np.float32),
            ["width", "height"],
            label="before_mc",
        ).opts(**im_opts)
    )
    + regrid(
        hv.Image(
            Y_hw_chk.max("frame").compute().astype(np.float32),
            ["width", "height"],
            label="after_mc",
        ).opts(**im_opts)
    )
)

# find cells

In [ ]:
ds = open_minian(intpath, return_dict=True)
Y_fm_chk = ds['Y_fm_chk']
opts_im = {'frame_width': 600, 'frame_height': 600, 'cmap': 'gray'}

In [ ]:
param_find_seed['wnd_k0'] = 6
param_find_seed['diff_thres'] = 8
param_find_seed

In [ ]:
max_proj = save_minian(
    Y_fm_chk.mean("frame").rename("max_proj"), intpath, overwrite=True
).compute()
seeds = find_seed(max_proj, **param_find_seed)
visualize_seeds(max_proj, seeds)

In [ ]:
param_constructA['max_size'] = 20 * 20
param_constructA['min_size'] = 6 * 6
param_constructA['me_thres'] = 0.5
param_constructA

In [ ]:
A = constructA(seeds, max_proj, **param_constructA)
visualize_seeds(max_proj, seeds) + plotA_contour(A, max_proj, im_opts=opts_im)

In [ ]:
A_merge = mergeA(A, **param_mergeA).rename("A")
plotA_contour(A_merge, max_proj, im_opts=opts_im)